In [1]:
import pandas as pd
# Load the Excel file
kundendatei = pd.read_excel("./outputs/Kundendatei_mapped.xlsx")
# clean Gewerk column
kundendatei['Gewerk'] = kundendatei['Gewerk'].replace({
    'Elektro Blitzschutz': 'Elektro',
    'Elektro NSP': 'Elektro',
    'Elektro Ladesäulen': 'Elektro',
    'RLT mit Hygiene': 'RLT',
    'Kältetechnik mit Hygiene': 'Kältetechnik',
})

In [2]:
gewerke = kundendatei['Gewerk'].unique().tolist()
gewerke


['RLT',
 'Elektro',
 'Haustechn Einricht',
 'Sanitär',
 'BuES, GLT, IZS',
 'Brandschutz',
 'Kältetechnik',
 'Heizung',
 'Türen und Tore',
 'Aufzug',
 'GEV',
 'DLÜA']

In [9]:
import json
mapped_structure_file = "./outputs/mapped_structure.json"
with open(mapped_structure_file, 'r') as file:
    mapped_structure = json.load(file)
mapped_structure

{'RLT': {'id': [430],
  'subcategories': [{'31211': {'Lüftungsanlagen': ['Schutzraumlüft. ohne Sandfilt.',
      'Zu-/Abluftanlage',
      'Lüftungsanlage mit Heizung',
      'Abluftanlage',
      'Schutzraumlüft. mit Sandfilter',
      'Lüftungsanlage Küche']}},
   {'31221': {'Zu-/Abluftgeräte': ['Abluftgerät']}},
   {'31223': {'Ventilatoren': ['Wand-/Fensterventilator',
      'Deckenventilator']}},
   {'31217': {'übergeordneter Schaltschrank': ['übergeordneter Schaltschrank']}},
   {'3123': {'Brandschutzklappen': ['Brandschutzklappen']}}]},
 'Elektro': {'id': [440, 450, 470],
  'subcategories': [{'3330': {'Blitzschutz-/Erdungsanlage': ['Blitzschutz/Erdung Gebäude']}},
   {'33103': {'Niederspannungsleitungsanlagen': ['Niederspannungsleitungsanlage']}},
   {'3410': {'Innenbeleuchtungseinrichtungen': ['Innenbel.einr. ortsf. Leuchten']}},
   {'3420': {'Außenbeleuchtungseinrichtungen': ['Außenbel.einr. ortsf. Leuchten']}},
   {'3450': {'Sicherheitsleuchten, einzelne': ['Sicherheitsl.einze

# gewerk > EQ-Klasse-Bezeichnung > Anlagenausprägung > EQ-Bezeichnung

In [6]:
from collections import defaultdict

def group_by_quotient(numbers, divisor=10):
    quotient_groups = defaultdict(list)
    for num in numbers:
        key = (num // divisor) * 10
        quotient_groups[key].append(num)
    # Sort by quotient for cleaner output
    return {key: sorted(quotient_groups[key]) for key in sorted(quotient_groups)}

In [18]:
for gebaude in kundendatei['WirtEinh'].unique():
    used_id = []   
    # create empty dataframe like EP_katalog
    ep_katalog = pd.DataFrame(columns=['OZ', 'Art', 'Menge', 'Einheit', 'Kurztext', 'Artikelnummer'])
    # Iterate through each Gewerk and create a new row in the EP_katalog
    major_oz = '01.'
    first_row = {
        'OZ': major_oz,
        'Art': 'NG - Normalgruppe',
        'Menge': None,
        'Einheit': None,
        'Kurztext': 'Wartung und Inspektion',
        'Artikelnummer': None
    }
    ep_katalog = pd.concat([ep_katalog, pd.DataFrame([first_row])], ignore_index=True)
    for i, gewerk in enumerate(gewerke):
        gewerk_df = kundendatei[kundendatei['Gewerk'] == gewerk]
        idx_gewerk = f"{i+1:02d}."
        for g in mapped_structure.keys():
            for id_gewerk in mapped_structure[g]['id']:
                if id_gewerk in used_id:
                    continue
                else:
                    used_id.append(id_gewerk)
                    break
            break
        new_row = {
            'OZ': f"{major_oz}{idx_gewerk}",
            'Art': "NG - Normalgruppe",
            'Menge': None,
            'Einheit': None,
            'Kurztext': f"{id_gewerk} - {gewerk}",
            'Artikelnummer': None
            }
        ep_katalog = pd.concat([ep_katalog, pd.DataFrame([new_row])], ignore_index=True)
        eq_klassen = gewerk_df['EQ-Klasse'].unique().tolist()
        for eq_klasse in eq_klassen:
            eq_klasse_df = gewerk_df[gewerk_df['EQ-Klasse'] == eq_klasse]
            eq_klasse_bz = eq_klasse_df['EQ-Klasse-Bezeichnung'].unique().tolist()[0]
            idx_eq_klasse = f"{i+1:02d}.{len(eq_klassen)+1:02d}."
            new_row = {
                'OZ': f"{major_oz}{idx_gewerk}{idx_eq_klasse}",
                'Art': "NG - Normalgruppe",
                'Menge': None,
                'Einheit': None,
                'Kurztext': f"{eq_klasse} - {eq_klasse_bz}",
                'Artikelnummer': None
            }

            ep_katalog = pd.concat([ep_katalog, pd.DataFrame([new_row])], ignore_index=True)
ep_katalog


,OZ,Art,Menge,Einheit,Kurztext,Artikelnummer
0,01.,NG - Normalgruppe,None,None,Wartung und Inspektion,None
1,01.01.,NG - Normalgruppe,None,None,430 - RLT,None
2,01.01.01.03.,NG - Normalgruppe,None,None,432 - Lüftungsanlagen,None
3,01.01.01.03.,NG - Normalgruppe,None,None,439 - Zu-/Abluftgeräte,None
4,01.02.,NG - Normalgruppe,None,None,430 - Elektro,None
5,01.02.02.06.,NG - Normalgruppe,None,None,446 - Blitzschutz-/Erdungsanlage,None
6,01.02.02.06.,NG - Normalgruppe,None,None,449 - Niederspannungsleitungsanlagen,None
7,01.02.02.06.,NG - Normalgruppe,None,None,445 - Innenbeleuchtungseinrichtungen,None
8,01.02.02.06.,NG - Normalgruppe,None,None,452 - Firmenlogos,None
9,01.02.02.06.,NG - Normalgruppe,None,None,472 - Küchen-/Kochgeräte,None


# Unfinished due to lack of time..